### Build and push image

In [3]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-pt-profiler

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email --registry-ids [763104351884,${account}])

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Sending build context to Docker daemon  10.75kB
Step 1/3 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.9.0-cpu-py38
1.9.0-cpu-py38: Pulling from pytorch-inference
f3ef4ff62e0d: Pulling fs layer
d25595960014: Pulling fs layer
7d0257ac10f7: Pulling fs layer
eb7d55e09844: Pulling fs layer
2ddbce0d4ab8: Pulling fs layer
47c9313de682: Pulling fs layer
d0219780fbef: Pulling fs layer
cc477482d2a4: Pulling fs layer
ae34c1ab11cf: Pulling fs layer
d5fd5cb88999: Pulling fs layer
0c8e7c9b1a7e: Pulling fs layer
186a4febd299: Pulling fs layer
ad1150ade9d0: Pulling fs layer
05ee92511d89: Pulling fs layer
7f34de5f4408: Pulling fs layer
72c133f8d094: Pulling fs layer
675dc66d6f82: Pulling fs layer
3f30eb65e5cf: Pulling fs layer
514ad8d70c7f: Pulling fs layer
c37521e4bf36: Pulling fs layer
491357517939: Pulling fs layer
eb7d55e09844: Waiting
2ddbce0d4ab8: Waiting
47c9313de682: Waiting
d0219780fbef: Waiting
cc477482d2a4: Waiting
ae34c1ab11cf: Waiting
d5fd5cb88999: Waiting
0c8e7c


An error occurred (InvalidParameterException) when calling the GetAuthorizationToken operation: Invalid parameter at 'registryIds' failed to satisfy constraint: 'Member must satisfy constraint: [Member must satisfy regular expression pattern: [0-9]{12}]'


In [5]:
import boto3

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name
image_uri = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-pt-profiler:latest".format(account_id,region)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


### Deploy our Model to an Endpoint
Our container has been pushed to ECR and our Model is in S3 now we have everything we need to Deploy to a SageMaker Endpoint.

In [6]:
import sagemaker
from sagemaker import get_execution_role, Session

sess = Session()

role = get_execution_role()

bucket = "sagemaker-sample-files"
key = "datasets/image/MNIST/model/pytorch-training-2020-11-21-22-02-56-203/model.tar.gz"

pt_model= "s3://{}/{}".format(bucket,key)

In [7]:
%%writefile inference.py

import json
import logging
import sys
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# defining model and loading weights to it.
def model_fn(model_dir): 
    model = Net()   
    with open(os.path.join(model_dir, "model.pth"), "rb") as f:
        model.load_state_dict(torch.load(f))
    model.to(device).eval()
    return model


# data preprocessing
def input_fn(request_body, request_content_type):
    assert request_content_type == "application/json"
    data = json.loads(request_body)["inputs"]
    data = torch.tensor(data, dtype=torch.float32, device=device)
    return data


# inference
def predict_fn(input_object, model):
    with torch.no_grad():
        prediction = model(input_object)
    return prediction


# postprocess
def output_fn(predictions, content_type):
    assert content_type == "application/json"
    res = predictions.cpu().numpy().tolist()
    return json.dumps(res)

Overwriting inference.py


In [9]:
client = boto3.client('codeguruprofiler')
pytorch_profiling_group_name = "SageMaker-PyTorch"


retrieve_latest_features_boto3_client_create_profiling_group = client.create_profiling_group(
    profilingGroupName=pytorch_profiling_group_name,

)


In [12]:
from sagemaker.pytorch import PyTorchModel
model = PyTorchModel(
    entry_point="inference.py",
    role=role,
    env={
    "PROFILING_GROUP_NAME": pytorch_profiling_group_name
    },
    model_data=pt_model,
    framework_version="1.9.0",
    image_uri=image_uri
)

In [13]:
%%time
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

instance_type = "ml.c4.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

-------!CPU times: user 467 ms, sys: 0 ns, total: 467 ms
Wall time: 3min 32s


In [16]:
%%time
import random
import numpy as np
for i in range (0,20000):
    dummy_data = {"inputs": np.random.rand(16, 1, 28, 28).tolist()}
    res = predictor.predict(dummy_data)

CPU times: user 16min 10s, sys: 7.39 s, total: 16min 17s
Wall time: 27min 39s
